# Predicting next word in a sentence 
## word here is represented by a character
Dataset is not published here, but for an idea, training is done on sentences that look like <b>AEDF234TH</b> and testing is done on sentences like <b>ASE452DG?</b>.

This is a standard LSTM implementation of this problem, using [Keras](https://keras.io/).

In [174]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import json

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation, Dropout
from tensorflow.keras.optimizers import Adam

np.random.seed(1)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Problem-specific data parameters
Each sentence in training data are of length :  `SENTENCE_LENGTH=9`

The vocabulary, `GRAMMAR` (or `GRAMMAR_MAP`, if the frequency of each word are taken into account) is given as :

`[A-Z][1-7]`

The prediction on the test data is the missing last word. So, in all the data sets, the **last** character in every sentence is taken as the `label` and the rest of it as `training` data. 

In [175]:
### Utils for this problem

SENTENCE_LENGTH = 9

ALPHABET = [chr(i) for i in range(65, 91)]
GRAMMAR_MAP = {}

for c in ALPHABET :
    GRAMMAR_MAP[c] = ord(c) - ord('A')

for i in range(1, 7+1) :
    GRAMMAR_MAP[str(i)] = GRAMMAR_MAP['Z'] + 1 + (ord(str(i)) - ord('1'))

REVERSE_GRAMMAR_MAP = ['']*len(GRAMMAR_MAP)

for char in GRAMMAR_MAP:
    REVERSE_GRAMMAR_MAP[GRAMMAR_MAP[char]] = str(char)
    
json.dumps(GRAMMAR_MAP)

'{"H": 7, "D": 3, "I": 8, "U": 20, "W": 22, "N": 13, "G": 6, "5": 30, "Q": 16, "1": 26, "V": 21, "4": 29, "3": 28, "J": 9, "O": 14, "K": 10, "Y": 24, "R": 17, "7": 32, "P": 15, "A": 0, "T": 19, "C": 2, "F": 5, "M": 12, "B": 1, "2": 27, "6": 31, "Z": 25, "X": 23, "S": 18, "E": 4, "L": 11}'

## Preprocessing the data

First, we will convert every sentence into a vector using One-Hot Encoding. 
Here, each word in the sentence is represented by a vector of n binary sub-vectors, where n is the number of different chars in the specified GRAMMAR (33, 26 alphabet + 7 numbers). 

Example:<br>
A becomes:<br>[**1**, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

1 becomes:<br>[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, **1**, 0, 0, 0, 0, 0, 0]

_ABCD1234_ becomes:<br>
[[<b>1</b>, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],<br>
  [0, <b>1</b>, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],<br>
 [0, 0, <b>1</b>, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],<br>
 [0, 0, 0, <b>1</b>, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],<br>
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, <b>1</b>, 0, 0, 0, 0, 0, 0,],<br>
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, <b>1</b>, 0, 0, 0, 0, 0],<br>
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, <b>1</b>, 0, 0, 0, 0],<br>
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, <b>1</b>, 0, 0, 0]]



Although not completely explored, one of the possibilities considered is to chunk the input vector into small groups. That is a sentence `ABCD1234`, if by default seen as collection of 

chunks of `1` word is represented as `[A, B, C, D, 1, 2, 3, 4]`

and the corresponding one-hot vector encoding will be of shape : `(8, 33)`

whereas if chunks of `2` words are considered, the sentence is represented as `[AB, CD, 12, 34]`

and the corresponding one-hot vector encoding will be of shape : `(4, (2,33))` or `(4, 66)` by flattening.

In [176]:
# Parameters

DECODED_PART_LENGTH = SENTENCE_LENGTH - 1
output_labels = len(GRAMMAR_MAP) # Number of output labels
CHUNK_LENGTH = 1

print("The input vector will have the shape {}x{}."
      .format(DECODED_PART_LENGTH//CHUNK_LENGTH, CHUNK_LENGTH*output_labels))

The input vector will have the shape 8x33.


## Data location
If using Google colab, set `COLAB=True` below and update the `parent_dir` to the parent folder where the folder `data` is present.

BY default, the code runs locally and `parent_dir` is set to the **current** directory.

The `data` folder is expected to have atleast 2 files : `train.csv` and `answers.csv`

```
parent_dir
    |
    |
    --data
        |
        -- train.csv
        -- answers.csv      
```

In [177]:
parent_dir = '.'
COLAB = False
if COLAB :
    from google.colab import drive
    drive.mount('/content/drive')
    parent_dir = '/content/drive/My Drive/LSTM'
    
train_file = parent_dir+'/'+'data/train.csv'
test_file = parent_dir+'/'+'data/answers.csv'

## Data preparation library
A small library named `alienchat_dataset` is written to prepare the datasets of **train**, **validation (fixed : 20%)** and **test** from the provided data files. 

`alienchat_dataset` provides a dataset object `DataSet` which is initialized by providing the vocabulary and the length of each sentence (including the label).

Although it is just preparing respective one-hot encoding as described above, some experimental features are present in the library such as 
```
chunk_length (default : 1)
```

`DataSet` has a method `load_data` which expects the `train_file` and `test_file`, as described above.

There are some (experimental) optional parameters that can be provided to `load_data` like :

`blow_training_data (default : 0)` - to repeat the training data and shuffle, if it is really small

`patterns (default : False)` - when set to `True`, for every `sentence` of length `SENTENCE_LENGTH`, a sub-sentence of length (`1+SENTENCE_LENGTH//2`) is added. 

The idea of `patterns` is to see if we can learn from subsequences for some repeating grammar. The idea is not completely explored.

When used, the input vector is filled with `np.zeros(len(GRAMMAR))` for every word less than the `SENTENCE_LENGTH`, in this case : (`SENTENCE_LENGTH - (1+SENTENCE_LENGTH//2)`) 

The data set preparation can be explored here : [AlienChatDataSet.ipynb](AlienChatDataSet.ipynb)


In [178]:
from alienchat_dataset import DataSet

In [179]:
alienchat = DataSet(GRAMMAR_MAP, SENTENCE_LENGTH, chunk_length=CHUNK_LENGTH)\
            .load_data(train_file, test_file)

In [180]:
print('Training data : ', alienchat.train.data.shape, 'Training labels : ', alienchat.train.labels.shape)
print('Validation data : ', alienchat.validation.data.shape, 'Validation labels : ', alienchat.validation.labels.shape)
print('Test data : ', alienchat.test.data.shape, 'Test labels : ', alienchat.test.labels.shape)


Training data :  (2034, 8, 33) Training labels :  (2034, 33)
Validation data :  (509, 8, 33) Validation labels :  (509, 33)
Test data :  (379, 8, 33) Test labels :  (379, 33)


## Building, training and evaluating the model

### Architecture
Sequential, at least 1 LSTM layer, Final Dense layer with Activation (Sigmoid)

**Parameter** `num_extra_hidden_layers` adds specified number of extra LSTM layers in between the first LSTM layer and the Dense layer.
#### Dropout
At least one Dropout layer for the last LSTM layer. Explored a bit with Dropout layers for each of the LSTM layers, if there were more.

**Number of neurons in each hidden layers** :
As a starting point, the following [formula](https://stats.stackexchange.com/a/136542) is referred :

$$N_h = \frac{N_s} {(\alpha * (N_i + N_o))}$$

$N_i$ is the number of input neurons, $N_o$ the number of output neurons, $N_s$ the number of samples in the trainings data, and $\alpha$ represents a scaling factor that is usually between 2 and 10. 

Alternatively, another simple rule, not necessarily optimal :

$$N_h = \frac{2} {3} *(N_i + N_o)$$

### Loss function
`categorical_crossentropy` : As mentioned [here](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy), it is suitable *"when there are two or more label classes"*
#### Metrics
As of [TF2.2+](https://github.com/tensorflow/tensorflow/blob/fabcd8f89cd5975331994049705e15cb75f32e0c/tensorflow/python/keras/engine/training.py#L463) documentation, using the string `acc` metrics automatically selects the relevant accuracy suitable to the loss function used. 
### Optimizer
`Adam` : variant of classic SGD, first described [here](https://arxiv.org/abs/1412.6980) but commonly known to adaptively vary learning rates.

### Summary

The best `test` accuracy of the model achieved as the following : `

| Extra Hidden layers      | Neurons / Hidden Layer | Epochs     | Train (loss / accuracy)| Validation (loss / accuracy)| Test (loss / accuracy)|
| :---        |    :----:   |    :----:   |    :----:   |    :----:   |    :----:   |
| 1      | Low (alpha=1)       | Moderate (50)   | 2.1047 / 34.96% | 2.0883 / 32.61% | 1.0338 / <b>88.13%</b>|


The best `train` accuracy of the model achieved, with a poor validation accuracy and lower `test` accuracy, as the following : `

| Extra Hidden layers      | Neurons / Hidden Layer | Epochs     | Train (loss / accuracy)| Validation (loss / accuracy)| Test (loss / accuracy)|
| :---        |    :----:   |    :----:   |    :----:   |    :----:   |    :----:   |
| 1      | Moderate (alpha=.5)       | High (100)   | 0.8386 / **70.35%** | 2.3840 / 37.33% | 0.9158 / <b>73.09%</b>|



In [181]:
# Build the model
print('Build model...')
#hidden_nodes = int(2/3 * (DECODED_PART_LENGTH * output_labels))

alpha = 1 # to 8

hidden_nodes = int(alienchat.train.data.shape[0] / (alpha * (DECODED_PART_LENGTH + output_labels)))

print("The number of hidden nodes is {}.".format(hidden_nodes))
num_extra_hidden_layers = 1
model = Sequential()
model.add(LSTM(hidden_nodes, return_sequences=(num_extra_hidden_layers>0), input_shape=(DECODED_PART_LENGTH//CHUNK_LENGTH, CHUNK_LENGTH*output_labels)))
model.add(Dropout(0.3))
if num_extra_hidden_layers > 0 :
    for _ in range(num_extra_hidden_layers-1) :
        model.add(LSTM(hidden_nodes, return_sequences=True))
        model.add(Dropout(0.3))
    model.add(LSTM(hidden_nodes))
    model.add(Dropout(0.3))
model.add(Dense(units=output_labels))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['acc'])

Build model...
The number of hidden nodes is 49.


In [182]:
batch_size=64
num_epochs=50
train_x = alienchat.train.data
train_y = alienchat.train.labels
validate_x = alienchat.validation.data
validate_y = alienchat.validation.labels
#_ = model.fit(train_x, train_y, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)
_ = model.fit(train_x, train_y, batch_size=batch_size, epochs=num_epochs, validation_data=(validate_x, validate_y))

Epoch 1/50
32/32 [==============================] - 1s 31ms/step - loss: 3.2737 - acc: 0.1391 - val_loss: 2.7948 - val_acc: 0.1670
Epoch 2/50
32/32 [==============================] - 0s 8ms/step - loss: 2.8222 - acc: 0.1421 - val_loss: 2.6713 - val_acc: 0.1670
Epoch 3/50
32/32 [==============================] - 1s 19ms/step - loss: 2.7194 - acc: 0.1613 - val_loss: 2.6176 - val_acc: 0.1670
Epoch 4/50
32/32 [==============================] - 0s 15ms/step - loss: 2.6583 - acc: 0.1608 - val_loss: 2.5602 - val_acc: 0.2200
Epoch 5/50
32/32 [==============================] - 0s 14ms/step - loss: 2.6017 - acc: 0.1834 - val_loss: 2.5083 - val_acc: 0.2181
Epoch 6/50
32/32 [==============================] - 0s 14ms/step - loss: 2.5590 - acc: 0.2080 - val_loss: 2.4583 - val_acc: 0.2240
Epoch 7/50
32/32 [==============================] - 0s 14ms/step - loss: 2.5155 - acc: 0.2050 - val_loss: 2.4217 - val_acc: 0.2299
Epoch 8/50
32/32 [==============================] - 0s 15ms/step - loss: 2.4995 - ac

In [183]:
test_loss, test_acc = model.evaluate(alienchat.test.data, alienchat.test.labels, verbose=2)

12/12 - 0s - loss: 0.8571 - acc: 0.8760


Curious about the wrongly predicted cases, the second best prediction is compared to the actual label. Turns out, from the first glimpse, the second best prediction was quite often the correct label.

In [187]:

def secondmax(arr) :
    b = np.sort(arr)
    return np.where(arr == b[-2])[0][0]


In [188]:
test = alienchat.test.rawdata
predictions = model.predict(alienchat.test.data)
test["predicted_word"] = [REVERSE_GRAMMAR_MAP[np.argmax(prediction)] for prediction in predictions]
test["second_best_pred"] = [REVERSE_GRAMMAR_MAP[secondmax(prediction)] for prediction in predictions]
#print(validate['predicted_word'])
test["encoded_word"] = test["sentence"].apply(lambda x : x[-1])
test[["sentence","predicted_word","encoded_word", "second_best_pred"]][test["encoded_word"] != test['predicted_word']].head(10)

,sentence,predicted_word,encoded_word,second_best_pred
15,2ESSE12S1,E,1,1
16,6MSSSG2M2,M,2,2
20,6YK2LLLZT,I,T,T
42,2G2GSS1SR,S,R,R
45,SRSRSRS4R,P,R,R
51,GRS2GRSGR,G,R,R
65,MMMMSSSSR,S,R,R
67,K2S1SEWEP,R,P,P
77,SM2SKZESP,E,P,S
78,SRSRSG2M2,M,2,2
